# Task 2: Recommendation Engine

This task is focused on creating recommendations for a user viewing a particular listing. Based on the presently viewed listing, top _k_ recommendations will be produced. This notebook explores two methodologies based on certain underlying assumptions and provides intuition behind future extension.

## Setting up the Notebook

We work on a running assumption that our task is directed to recommend listings to a user on a search engine/forum/website targeted towards house listings. 

In [ ]:
import numpy as np
import pandas as pd
import random
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cdist

## Load the Data

Here, we load the input data for this notebook. There are two identically-indexed dataframes that we work off, _df_pristine_ being the displayable dataframe, while _df_numerical_ is the one used for performing calculations below. Note that _df_pristine_ is the original dataset provided with Task 1, cleaned of invalid values and outliers. It has not been transformed in any other way, and does not have any handling for filling in missing values as based on our assumed task, we take our directory of listings as a given and do not wish to externally modify it.

Note that _df_pristine_ and _df_numerical_ are required to have the same number of rows, although their columns might differ in both number and data. 

In [ ]:
RANDOM_CONTROL = 42 # For reproducibility.

df_pristine = pd.read_csv('data/train.csv') # We display this.
df_numerical = pd.read_csv('data/processed.csv') # We work on this.

#df_pristine.shape
#df_numerical.shape

## Computing the Top Recommendations

Here, we create the top-k recommendations for a given row from _df_pristine_. There are two approaches explored in this notebook:

1. **Cold Start**

This approach is motivated towards figuring out recommendations for a user without any prior browsing history. The inherent problem statement thus translates to: 'Given a listing, find top-k recommendations from the dataset for that listing', being an extension of recommendations based on _item-item similarity_.

Using the task-specific knowledge, our approach is directed by 4 factors: **geographical**, **tangible**, **intangible** and **random**. The assumption in this approach is captured by our intuition that any user viewing a listing will be influenced by the above 4 factors.

**Geographical** A user searching for a listing is possibly motivated by the surrounding geography. They might be interested in listings in nearby locations and thus recommending them would make meaningful sense. This is captured by the multi-variate vector of location and price, wherein listings in similar locations with similar pricing would make for good recommendation. [**Canberra** distance](https://en.wikipedia.org/wiki/Canberra_distance) measure is used to compare similarity as it allows for weighting across different scales for coordinates and prices.

**Tangible** A user is possibly motivated by salient features of a listing. They might be looking to buy a family home or interested in moving into a new bungalow or even looking to rent a small space. These are tangible features that a user has a mental image for and are captured as a range of descriptors like area of property, type of property, number of accompanying beds/bathrooms which can be explicitly identified. Since it is a multi-dimensional vector where we wish to inflate the influence of property types, we use the [Cosine distance](https://en.wikipedia.org/wiki/Cosine_similarity) to capture similarity.

**Intangible** A user might be interested in available facilities and amenities for their desired listing wherein parents would be more motivated by the closeness to schools while party-goers might prefer downtown locations. Transport infrastructure is another factor that might influence decisions regarding certain properties. These are captured by the auxiliary dataset presented to us in Task 1 and encoded here in the form of distance to nearest MRT, schools or shopping malls. Since this works off distance measures to nearest amenities, we use the [Manhattan distance](https://en.wikipedia.org/wiki/Taxicab_geometry) here to capture similarity across listings.

**Random** A recommendation made based on a single listing is likely to be overspecialized as it does not have much to go on other than the lone listing. There is always a chance that the user visited this listing in error or their interests lie orthogonal to the features of the current listing, which can be solved by randomizing the recommendations so as not to silo the user into a subset of the dataset. [**Chebyshev** distance](https://en.wikipedia.org/wiki/Chebyshev_distance) is used to compare similarity to account for random listings in a similar price range.

Our goal is to create a set of recommendations that best captures all the factors outlined above. As such, we first custom-define hyperparameters of interest per factor. Geographical and tangible factors are given a weight of 1.0 each, intangible factors are weighted at 0.5 and random factors are assigned a weight of 0.1. Based on this, we create a collection of recommendations and then sample _k_ recommendations from that collection to display to the user. As a result, this process is not deterministic and allows for tuning of factors in the long run by observing user link click behaviour.

2. **Browsing History**

This approach is motivated towards figuring out recommendations for a user with a prior browsing history. The inherent problem statement thus translates to: 'Given a user with a viewing history, find top-k recommendations from the dataset for the listing they are currently viewing', being an extension of recommendations based on _user-item similarity_. 

We reference the browsing history, augmenting it with the freshly viewed listing. A view is implicitly assumed to be a "rating" by the user for a particular listing as a user is more likely to browse listings they are interested in or curious about. Since we have the numerical representation of each listing, we can create an aggregation of user weights for each listing(mean being the representative user value for each feature) and then calculate the listing in our dataset most similar to the user profile.

**Decay** A decay factor is used to control for freshness of profile. As a user browses through a set of listings, it is assumed they are getting closer to what they wish to search for. As a result, the newer listings are weighted exponentially more than the older ones. We use a factor of **0.75** to control for the 5^{th} listing in the past being roughly a quarter as relevant as the present listing. This is presented as a parameter that can be tuned. 

In [ ]:
# To optimize performance, calculate sorted similarity matrix here based on slices as mentioned above.
# For a persistent application, this can be factored out into a background batch job run at frequent intervals,
# to constantly maintain an up-to-date set of listing.

# Geographical slice
geo_slice_cols = ('lat', 'lng', 'price')
np_geo = df_numerical.loc[:,geo_slice_cols].to_numpy()
sim_geo = pd.DataFrame(cdist(np_geo, np_geo, metric='canberra'))
sim_geo_sorted = sim_geo.columns.to_numpy()[np.argsort(sim_geo.to_numpy())]

# Tangible slice
tangible_slice_cols = ('num_baths', 'num_beds', 'size_sqft', 'bungalow', 'condo', 'hdb', 'corner', 'landed', 'protected', 'terraced house', 'walk-up')
np_tangible = df_numerical.loc[:,tangible_slice_cols].to_numpy()
sim_tangible = pd.DataFrame(cdist(np_tangible, np_tangible, metric='cosine'))
sim_tangible_sorted = sim_tangible.columns.to_numpy()[np.argsort(sim_tangible.to_numpy())]

# Intangible slice
intangible_slice_cols = ('dist_2_nearest_mrt', 'dist_2_nearest_sm', 'dist_2_nearest_ps', 'dist_2_nearest_ss', 'dist_2_nearest_cc')
np_intangible = df_numerical.loc[:,intangible_slice_cols].to_numpy()
sim_intangible = pd.DataFrame(cdist(np_intangible, np_intangible, metric='cityblock'))
sim_intangible_sorted = sim_intangible.columns.to_numpy()[np.argsort(sim_intangible.to_numpy())]

# Random slice
random_slice_cols = ('price')
np_random = df_numerical.loc[:,random_slice_cols].to_numpy().reshape(-1,1)
sim_random = pd.DataFrame(cdist(np_random, np_random, metric='chebyshev'))
sim_random_sorted = sim_random.columns.to_numpy()[np.argsort(sim_random.to_numpy())]

In [ ]:
def get_top_recommendations(row, **kwargs) -> pd.DataFrame:
    
    #####################################################
    ## Initialize the required parameters
    k = None
    cold_start = True
    df_history = None
    geo_factor = 1.0
    tangible_factor = 1.0
    intangible_factor = 0.5
    random_factor = 0.1
    decay_factor = 0.75    
    
    # Extract all **kwargs input parameters
    # and set the used paramaters
    for key, value in kwargs.items():
        if key == 'k':
            k = value
        elif key == 'cold_start':
            cold_start = value
        elif key == 'browsing_history':
            df_history = value
        elif key == 'geo_factor':
            geo_factor = value
        elif key == 'tangible_factor':
            tangible_factor = value
        elif key == 'intangible_factor':
            intangible_factor = value
        elif key == 'random_factor':
            random_factor = value
        elif key == 'decay_factor':
            decay_factor = value
    #####################################################
    ## Compute recommendations
    df_result = []
    idx = df_pristine.loc[df_pristine['listing_id'] == row['listing_id']].index[0]

    geo_topk = int(np.ceil(geo_factor * k))
    tangible_topk = int(np.ceil(tangible_factor * k))
    intangible_topk = int(np.ceil(intangible_factor * k))
    random_topk = int(np.ceil(random_factor * k))
    dupe_removed = 0

    # No past viewing history available. Recommend based on current listing.
    if cold_start:
        # Geographical Slice
        sim_geo_topk = pd.DataFrame(sim_geo_sorted[:, 0:2*geo_topk:1])
        for i in range(geo_topk):
            rec = sim_geo_topk[i][idx]
            if row.equals(other=df_pristine.iloc[rec]):
                dupe_removed += 1
                continue            
            df_result.append(df_pristine.iloc[rec])
        i = geo_topk
        while dupe_removed > 0:
            rec = sim_geo_topk[i][idx]
            i += 1
            if row.equals(other=df_pristine.iloc[rec]):
                continue
            df_result.append(df_pristine.iloc[rec])
            dupe_removed -= 1
                        
        # Tangible Slice
        sim_tangible_topk = pd.DataFrame(sim_tangible_sorted[:, 0:2*tangible_topk:1])
        for i in range(tangible_topk):
            rec = sim_tangible_topk[i][idx]
            if row.equals(other=df_pristine.iloc[rec]):
                dupe_removed += 1
                continue            
            df_result.append(df_pristine.iloc[rec])
        i = tangible_topk
        while dupe_removed > 0:
            rec = sim_tangible_topk[i][idx]
            i += 1
            if row.equals(other=df_pristine.iloc[rec]):
                continue
            df_result.append(df_pristine.iloc[rec])
            dupe_removed -= 1
        
        # Intangible Slice
        sim_intangible_topk = pd.DataFrame(sim_intangible_sorted[:, 0:2*intangible_topk:1])
        for i in range(intangible_topk):
            rec = sim_intangible_topk[i][idx]
            if row.equals(other=df_pristine.iloc[rec]):
                dupe_removed += 1
                continue            
            df_result.append(df_pristine.iloc[rec])
        i = intangible_topk
        while dupe_removed > 0:
            rec = sim_intangible_topk[i][idx]
            i += 1
            if row.equals(other=df_pristine.iloc[rec]):
                continue
            df_result.append(df_pristine.iloc[rec])
            dupe_removed -= 1
        
        # Random Slice
        sim_random_topk = pd.DataFrame(sim_random_sorted[:, 0:2*random_topk:1])
        for i in range(random_topk):
            rec = sim_random_topk[i][idx]
            if row.equals(other=df_pristine.iloc[rec]):
                dupe_removed += 1
                continue            
            df_result.append(df_pristine.iloc[rec])
        i = random_topk
        while dupe_removed > 0:
            rec = sim_random_topk[i][idx]
            i += 1
            if row.equals(other=df_pristine.iloc[rec]):
                continue
            df_result.append(df_pristine.iloc[rec])
            dupe_removed -= 1
                    
        df_result = random.sample(df_result, k)

    # Browsing history is available. Utilize it to recommend based on user-item similarity.
    else:
        df_history = pd.concat([df_history, row.to_frame().T])
        user_profile = pd.DataFrame()
        decay = 1
        # Iterate history from newer views to older views, accounting for decay
        for idx in reversed(df_history.index):
            user_profile = pd.concat([user_profile, decay*df_numerical.loc[idx].to_frame().T]) 
            decay *= decay_factor
        user_prefs = user_profile.mean().to_frame().T
        sim_listings = pd.DataFrame(cosine_similarity(user_prefs, df_numerical))
        sim_listings_topk = sim_listings.columns.to_numpy()[np.argsort(sim_listings.to_numpy())][:, -1:-k-1:-1]

        for i in range(k):
            rec = sim_listings_topk[0][i]
            df_result.append(df_pristine.iloc[rec])
            
    # Return the dataset with the k recommendations
    df_result = pd.concat(df_result)
    df_result = pd.DataFrame(df_result.values.reshape(k, 21))
    df_result.columns = list(df_pristine.columns)
    return df_result


## Testing the Recommendation Engine

Here, you can test for different recommendations created by this notebook.

### Pick a Sample Listing as Input

In [ ]:
# Pick a row id of choice
#row_id = 10
row_id = 20
#row_id = 30
#row_id = 40
#row_id = 50

# Get the row from the dataframe 
row = df_pristine.iloc[row_id]

# Just for printing it nicely, we create a new dataframe from this single row
pd.DataFrame([row])

## Compute and Display the recommendations

Here you can get different recommendations based on your parameters. _k_ specifies the number of recommendations you want, _cold_start_ allows you to switch between the two approaches and _num_sig_history_ allows you to define the amount of browsing history you consider sufficient to allow the second approach to work.

In [ ]:
k = 10 # Specify how many recommendations you want

cold_start = True # Specify True if new user; else assume browsing history exists
num_sig_history = 5 # Specify number of viewings that define a "significant" browsing history

if cold_start:
    df_recommendations = get_top_recommendations(row, k=k, cold_start=cold_start, geo_factor=1.0, tangible_factor=1.0, intangible_factor=0.5, random_factor=0.1)
else:
    df_history = df_pristine.sample(n=num_sig_history, axis=0, random_state=RANDOM_CONTROL, ignore_index=False)
    df_recommendations = get_top_recommendations(row, k=k, cold_start=cold_start, browsing_history=df_history, decay_factor=0.75)
    
df_recommendations.head(k)

## Analysing the approach

**On filtering**

While we work off slices of the original dataset, we never filter it based off attribute values. Since listings in a particular location or of a particular type could easily be viewed by a user via filtering, we do not replicate that functionality here. In the interest of non-trivial educational experience, we decide to explore similarity measures instead in our methodology.

**On collaborative filtering**

While the constrained nature of the notebook makes it a tough ask to simulate collaborative filtering, we wish to emphasize that we do not believe it makes for a good approach in this setting. A house listing is highly unlikely to garner ratings or reviews from users, and the ones that do are expected to be listings that would be taken down as the user is likely to close the deal. Unlike marketplaces with multiple copies of items, or websites with multiple movies, a house listing is almost always unique with different units in the same building often going for vastly varied prices. We are unable to figure out a mechanism where a collaborative filtering approach can be justified and as such, it is not further explored.

**On dynamic tuning**

We have not implemented any dynamic tuning of the hyper-parameters we talked about however we wish to motivate a discussion on the merits of doing that. As recommendations created via cold-start are explainable, i.e. we can tell which of the factors generated that particular recommendation, it is feasible to track user behaviour for all clicks in the recommendation space and the proportion of clicks for a particular feature. This allows for a dynamic macro-level tuning of the factors based on what the general user base prefers to serve better recommendations.

**Overspecialization**

While cold-start tends to overspecialize in its recommendations, randomness does help in mitigating it. Do note however that the effects of over-specialization are much more pronounced in the case of browsing history as the user profile drives recommendations here and it is much more likely for a user to be siloed in to a subset of the dataset. While we use the decay factor to tail off the effect of stale viewings, we do note that overspecialization might be intended behaviour in this setting. Unlike settings where users are assumed to always return, like Netflix or social networks, a search for house is usually much more constrained and comes with an end goal in mind wherein a user is motivated to seek out their favoured house as quickly as possible. In such a scenario, it might help to narrow the search space of the user via overspecialization to speed up the process. To mitigate the effects of missing a relevant listing, we can additionally show some random listings to serve as potential hooks out of the silo for the user. If the user clicks on one such listing, the decay factor should allow to freshen up the recommendations. This functionality has not been simulated here due to the constrained nature of this notebook.

**Applying cold start to browsing history**

Since we wished to explore the implementation of _user-item similarity_ in our project, we decided not to use this approach. However, it is possible to extend the first methodology to account for browsing history, wherein we generate cold-start recommendations based on each listing in the browsing history, with the decay factor serving as the weight for each listing by which we eventually sample _k_ recommendations to show to the user. In practice, we find this approach outperforms _user-item similarity_ as implemented here although we believe this might be due to inadequacies in the way we generate the user profile.